In [1]:
import pandas as pd
import glob

# Загружаем все .csv файлы из текущей папки
files = glob.glob("*.csv")

# Читаем все файлы в список таблиц
dfs = [pd.read_csv(f) for f in files]

# Склеиваем всё в один большой DataFrame
df = pd.concat(dfs, ignore_index=True)

# Проверим результат
print(f"Считано файлов: {len(files)}")
print(f"Всего строк: {len(df)}")
df.head()

Считано файлов: 32
Всего строк: 6142


,ax,ay,az,gx,gy,gz,label
0,-0.061,0.287,1.162,3.183,12.038,1.588,down
1,0.024,0.212,0.878,-18.191,17.618,-13.366,down
2,0.026,0.206,0.884,39.656,-8.191,14.679,down
3,-0.029,0.287,1.156,26.275,4.626,6.229,down
4,0.011,0.369,1.163,6.885,-1.817,7.275,down


In [2]:
import numpy as np

WINDOW_SIZE = 50
X = []
y = []

for i in range(0, len(df) - WINDOW_SIZE, WINDOW_SIZE):
    window = df.iloc[i:i+WINDOW_SIZE]
    label = window['label'].mode()[0]  # Самая частая метка
    features = window.drop('label', axis=1).values.flatten()
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)

print("Готово окон:", len(X))

Готово окон: 122


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier()
model.fit(X_train, y_train)

preds = model.predict(X_test)
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

        down       0.78      0.88      0.82         8
        idle       1.00      1.00      1.00         3
        left       1.00      0.67      0.80         3
       right       1.00      1.00      1.00         1
          up       0.80      0.80      0.80        10

    accuracy                           0.84        25
   macro avg       0.92      0.87      0.88        25
weighted avg       0.85      0.84      0.84        25



In [4]:
from micromlgen import port

c_model = port(model)
with open("gesture_model.h", "w") as f:
    f.write(c_model)


In [5]:
import joblib
joblib.dump(model, "model.pkl")

['model.pkl']